## Verity Cloud API

In [ ]:
import requests

VERITY_API_URL = "https://us-central1-verity-exam.cloudfunctions.net/"

# From the profile page in Verity, click on "API Key", then paste it here.
API_KEY = "eyhjbGcaklaj3ovkg.2wij3uiwiblahblahblah"

def cloudFunction(name, data={}):
  r = requests.post(VERITY_API_URL + name,
                    headers={
                      'Content-Type': 'application/json',
                      'Authorization': 'Bearer ' + VERITY_API_KEY,
                    },
                    json={"data": data},
                    )
  #print(r)
  j = r.json()
  #print(j)
  return j['result']

## Manual step-by-step example.

In [ ]:
examSpecs = cloudFunction('searchExamSpecs')
print(f"{examSpecs=}")
print(f"{examSpecs.keys()=}")

In [ ]:
examSpecId = "-NF_H1"
examExecId = cloudFunction('startExam', data={"exam_spec_id": examSpecId})
print(f"{examExecId=}")

In [ ]:
responseId, questionText = cloudFunction('getQuestion', data={"exam_exec_id": examExecId})
print(f"{responseId=}")
print(f"{questionText=}")

In [ ]:
score = cloudFunction('submitResponse', data={"exam_exec_id": examExecId, "response_id": responseId, "response_text": "The name of the three-headed dog that guards the underworld is Cerberus."})
print(f"{score=}")

## Local student model

In [ ]:
# Load local huggingface model
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")
model.config.pad_token_id = model.config.eos_token_id

print(f"Loaded {MODEL_NAME=}")

In [ ]:
def complete(prompt, n=1, engine=None, max_tokens=40, logprobs=None, logit_bias=None, temperature=0.7):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda')
    generated_ids = model.generate(input_ids, do_sample=True, max_new_tokens=max_tokens, num_return_sequences=n, use_cache=False, temperature=temperature)
    c = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    
    # Clean up the output (may need to be customized for various models...)
    c = [x[len(prompt):] for x in c]  # Skip the prompt
    c = [x.strip() for x in c]  # Trim whitespace from the ends
    c = [x.split('\n')[0] for x in c]    # Stop at the first stop character -- TODO: figure out how to truncate generation!!!
    c = [x.strip() for x in c]   # Strip whitespace
    return c

## Cloud student model

In [ ]:
import requests
import tenacity

MODEL_API_URL = 'https://api.openai.com/v1/'
MODEL_API_KEY = 'sk-YOUR_OPEN_AI_KEY'

## Standard OpenAI-style query
#def complete(prompt, n=1, engine='davinci', max_tokens=20, temperature=0.0, logprobs=None, logit_bias={}):
#    r = requests.post(MODEL_API_URL + 'engines/' + engine + '/completions',
#                      headers={
#                        'Content-Type': 'application/json',
#                        'Authorization': 'Bearer ' + MODEL_API_KEY,
#                      },
#                      json={
#                        'prompt': prompt,
#                        'max_tokens': max_tokens,
#                        'stop': '\n',
#                        'n': n,
#                        'temperature': temperature,
#                        'logprobs': logprobs,
#                        'logit_bias': logit_bias,
#                      })
#    j = r.json()
#    #print(j)
#    return [c['text'] for c in j['choices']]

@tenacity.retry(wait=tenacity.wait_random_exponential(min=1, max=20), stop=tenacity.stop_after_attempt(3))
def chatComplete(messages, n=1, engine='gpt-3.5-turbo', max_tokens=20, temperature=0.7):
    r = requests.post(API_URL + 'chat/completions',
                      headers={
                        'Content-Type': 'application/json',
                        'Authorization': 'Bearer ' + API_KEY,
                      },
                      json={
                        'model': engine,
                        'messages': messages,
                        'max_tokens': max_tokens,
                        'n': n,
                        'temperature': temperature,
                      })
    j = r.json()
    print(j)
    #print(j['choices'])
    return [c['message']['content'] for c in j['choices']]

def complete(prompt, n=1, max_tokens=20, temperature=0.7):
    return chatComplete([{"role": "system", "content": prompt}], n=1, max_tokens=max_tokens, temperature=0)

## Automated exam taking

In [ ]:
complete("Q: How many ounces in a pound.\nA:")

In [ ]:
# Exams to take
examSpecIds = [
    '-NF_H1',                # Greek mythology
    '-NYry2Ic0-WhbvgiiSh2',  # Sailing small boats
    '-NF_T1',                # Pokemon 
]  

for examSpecId in examSpecIds:
    examExecId = cloudFunction('startExam', data={"exam_spec_id": examSpecId})
    while True:
        responseId, questionText = cloudFunction('getQuestion', data={"exam_exec_id": examExecId})
        if responseId == 'COMPLETED':
            print("Total score:", questionText)
            break
        prompt = ""
        prompt += "You are a student taking an exam. Please respond clearly and accurately to the following question, and provide supporting details when possible.\n"
        prompt += "Question: " + questionText + "\n"
        prompt += "Answer:"
        print(f"{prompt=}")
        response = complete(prompt)[0]
        print(f"{response=}")
        score = cloudFunction('submitResponse', data={"exam_exec_id": examExecId, "response_id": responseId, "response_text": response})